<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [0]:
!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

In [0]:
!unzip concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

## Import Libraries and Packages

In [3]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.applications.vgg16 import preprocess_input

Using TensorFlow backend.


## Define Global Constants

In [0]:
num_classes = 2
image_resize = 224
batch_size = 100

## Create ImageDataGenerator Instances

In [0]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [6]:
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [7]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size,
    class_mode='categorical')

Found 9501 images belonging to 2 classes.


## Build, Compile and Fit Model

In [0]:
model = Sequential()

In [9]:
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

58892288/58889256 [==============================] - 2s 0us/step


In [0]:
model.add(Dense(num_classes, activation='softmax'))

In [0]:
model.layers[0].trainable = False

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2

In [15]:
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/2
301/301 [==============================] - 110s 364ms/step - loss: 0.1205 - accuracy: 0.9604 - val_loss: 5.1449e-04 - val_accuracy: 0.9928
Epoch 2/2
301/301 [==============================] - 100s 333ms/step - loss: 0.0252 - accuracy: 0.9949 - val_loss: 0.0665 - val_accuracy: 0.9960


<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [0]:
resnet_model = keras.models.load_model('./classifier_resnet_model.h5')

In [19]:
test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    shuffle = False)

Found 500 images belonging to 2 classes.


In [22]:
print('for VGG16')
performance_vgg = model.evaluate_generator(test_generator, verbose=1)
print('for ResNet50')
performance_resnet = resnet_model.evaluate_generator(test_generator, verbose=1)

for VGG16
16/16 [==============================] - 1s 86ms/step
for ResNet50
16/16 [==============================] - 2s 109ms/step


In [27]:
print('VGG16\n\t accuracy: {}, loss: {}'.format(performance_vgg[1], performance_vgg[0]))
print('-'*70)
print('ResNet50\n\t accuracy: {}, loss: {}'.format(performance_resnet[1], performance_resnet[0]))

VGG16
	 accuracy: 0.9959999918937683, loss: 0.0012676285114139318
----------------------------------------------------------------------
ResNet50
	 accuracy: 0.9620000123977661, loss: 0.07847046852111816


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [86]:
predictions_vgg = model.predict_generator(test_generator, verbose=1)
predictions_resnet = resnet_model.predict_generator(test_generator, verbose=1)

16/16 [==============================] - 2s 108ms/step


In [0]:
import numpy as np

In [0]:
def print_predictions(pred):
    for i in range(5):
        result = np.where(max(pred[i]))
        print('\t{}. '.format(i+1), end=' ')
        if result[0] == 0:
            print('Negative')
        else:
            print('Positive')

In [96]:
print('VGG16')
print_predictions(predictions_vgg)
print('-'*30)
print('ResNet50')
print_predictions(predictions_resnet)

VGG16
	1.  Negative
	2.  Negative
	3.  Negative
	4.  Negative
	5.  Negative
------------------------------
ResNet50
	1.  Negative
	2.  Negative
	3.  Negative
	4.  Negative
	5.  Negative


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).